Pada proses saya melakukan visualisasi dan analisis serta klastering data Nasabah German Credit.

# Intalasi serta import library dan package yang dibutuhkan pada proses ini

In [ ]:
pip install --upgrade category_encoders

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly as py
import os
import plotly.io as pio
pio.renderers.default='notebook'

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering, DBSCAN
import scipy.cluster.hierarchy as shc 
import plotly.graph_objects as go


import category_encoders as ce

plt.style.use('seaborn-colorblind')
%matplotlib inline

# Membaca data

In [ ]:
# read data into dataframe and showing 5 data
df =  pd.read_csv('../input/german-credit/german_credit_data.csv', index_col=0)
df.head(5)

**Deskripsi Data**

Pada dataset ini didapatkan :
* Terdapat 1000 row data dengan 9 kolom
* Masih ada data yang kosong yaitu pada kolom saving accounts dan checking accounts
* Terdapat 4 kolom dengan tipe integer dan 5 kolom dengan tipe data object

In [ ]:
df.info()

**Deskripsi dataset**

Pada tabel deskripsi dataset dibawah didapatkan nilai-nilai statistikal dari tiap kolom yaitu : 
* Count (Jumlah data) yaitu 1000 tiap-tiap kolomnya
* Mean (Nilai rataan) yaitu pada kolom Age=35.5, Job=1.9, Credit amount=3271.26, Duration=20.9
* Standar Deviasi yaitu pada kolom Age=11.38, Job=0.65, Credit amount=2822.74, Duration=12.06
* min (Nilai terendah) yaitu pada kolom Age=19, Job=0, Credit amount=250, Duration=4
* 25% atau Kuartil-1 pada data yaitu kolom Age=27, Job=2, Credit amount=1365, Duration=12
* 50% atau Kuartil-2 atau nilai tengah pada data yaitu kolom Age=33, Job=2, Credit amount=2319, Duration=18
* 75% atau Kuartil-3 pada data yaitu kolom Age=27, Job=2, Credit amount=1365, Duration=18
* max (Nilai tertinggi) yaitu pada kolom Age=75, Job=3, Credit amount=18424, Duration=72

In [ ]:
df.describe()

**Deskripsi dataset**

Deskripsi dataset untuk data yang bertipe data object yaitu : 
* Count (Jumlah data) yaitu 1000 tiap-tiap kolomnya kecuali pada kolom Saving accounts=817 dan kolom Checking account=606
* Unique (Jumlah data unik pada kolom) yaitu pada kolom Sex=2, Housing=3, Saving accounts=4, Checking account=3, Purpose=8
* Top (Data yang paling sering muncul) yaitu pada kolom Sex=male, Housing=own, Saving accounts=little, Checking account=little dan Purpose=car
* Freq (Frekuensi data top muncul) yaitu pada kolom Sex=690, Housing=713, Saving accounts=603, Checking account=274 dan Purpose=337.

In [ ]:
df.describe(include=['object'])

**Menampilkan nilai unik dari masing-masing kolom**

In [ ]:
print('Nilai Unik Sex/Gender:', df['Sex'].unique())
print('')
print('Nilai Unik Housing/Status Tinggal:', df['Housing'].unique())
print('')
print('Nilai Unik Saving accounts/Uang Tabungan:', df['Saving accounts'].unique())
print('')
print('Nilai Unik Checking account/Tabungan Giro:', df['Checking account'].unique())
print('')
print('Nilai Unik Checking Purpose/Tujuan', df['Purpose'].unique())

**Membagi data**

Pada proses ini saya membagi data yaitu data yang bertipe data categorical/object disimpan dalam variable categorical dan data yang bertipe data integer pada variable numeric.

In [ ]:
numeric = ['Age', 'Job', 'Credit amount', 'Duration']
categorical = ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']

# EDA

**Cek kembali data yang kosong**

Didapatkan pada kolom Saving account 183 data kosong dan pada kolom Checking amount 394 data kossong sedangakan pada kolom lainnya tidak terdapat data yang kosong.

In [ ]:
df.isnull().sum()

Melihat lagi data unik pada kolom saving accounts dan checking account karena terdapat data yang kosong pada ke-2 kolom tsb.

In [ ]:
print('Data Uik saving accounts:', df['Saving accounts'].unique())
print('Data Unik checking account:', df['Checking account'].unique())

Saya mengisi data yang kosong tersebut dengan label 'unknown' atau tidak diketahui

In [ ]:
df['Saving accounts'] = df['Saving accounts'].fillna('unknown')
df['Checking account'] = df['Checking account'].fillna('unknown')
df.head(5)

# Visualisasi 1

In [ ]:
df.hist(figsize = (20,15));

Pada visualisasi pertama ini membuat visualisasi grafik pada data numerik yaitu kolom age, job, credit amount dan duration. Dan didapatkan :
* Pada grafik age/umur terjadi pelonjakan kenaikan dari umur 20 ke 25 dan dari 25 sampai 40 merupakan umur yang dengan jumlah terbanyak dan semakin tua umurnya semakin menurun grafik
* Pada grafik job label 2 mendominasi dengan jumlah >600
* Pada grafik Credit amount paling banyak yaitu pada jumlah credit amount 0-3000 dan untuk semakin besar credit amount semakin sedikit jumlahnya juga
* Pada grafik duration didapatkan durasi yang paling banyak diambil oleh nasabah yaitu dengan durasi 2-25 bulan, untuk yang lebih dari itu juga ada tetapi terdapat perbedaan grafik yang cukup berbeda signifikan lebih sedikit.

In [ ]:
for col in df[categorical].columns:
    sns.countplot(y =col, data = df)
    plt.show()

Pada visualisasi pertama ini membuat visualisasi grafik pada data categorical yaitu kolom sex, housing, saving accounts, checking account dan purpose. Dan didapatkan :

* Pada atribut Sex/Gender : Nasabah kredit bank lebih banyak dari gender laki-laki perbandingannya mencapai 7:3.     
* Pada atribut Housing/Rumah : Nasabah kredit bank paling banyak sudah memiliki rumah sendiri dan perbandingannya cukup dominan 7:1:2 (milik:bebas:sewa).      
* Pada atribut Saving accounts/Jumlah tabungan : Nasabah kredit bank didominasi yang memiliki jumlah tabungan dengan jumlah sedikit.     
* Pada atribut Checking accounts/Jumlah tabungan Giro: Nasabah kredit bank didominasi dengan nasabah yang tidak diketahui statusnya.       
* Pada atribut Purpose : Nasabah kredit bank didominasi yang memiliki tujuan kredit yaitu untuk membeli mobil dan 3 tujuan tertinggi dibawahnya yaitu : radio/TV, furnitur/perlengkapan dan bisnis.

# Correlation Matrix

Pada proses ini saya menggunakan tabel grafik korelasi matriks unutk mengetahui korelasi antara tiap kolom/variable fitur. Terdapat kontras warna juga untuk mengetahui korelasi jika semakin berkorelasi warna nya akan semakin terang.

In [ ]:
corr = df.corr()
plt.figure(figsize=(10,8));
sns.heatmap(corr, annot=True, fmt='.2f');

In [ ]:
data = df.copy()

# Encode 

Pada proses ini dilakukan encode data yang mempunyai tipe data categorical/object menjadi integer.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
from sklearn.preprocessing import LabelEncoder
for label in categorical:
    data[label] = encoder.fit_transform(data[label])

In [ ]:
data[categorical]

# Normalisasi

Proses ini melakukan normalisasi data menggunakan Standard Scaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(X_scaled, columns=data.columns)
data_scaled.head()

# Klasterisasi

Pada proses ini dilakukan klasterisasi data

In [ ]:
X = data

**Mencari Nilai Kluster**

Pada proses mencari nilai K yang akan digunakan sebagai nilai kluster menggunakan intertia plot-elbow method, dan didapatkan nilai K=4

In [ ]:
#Elbow Method - Inertia plot
inertia = []
#looping the inertia calculation for each k
for k in range(1, 10):
    #Assign KMeans as cluster_model
    cluster_model = KMeans(n_clusters = k, random_state = 24)
    #Fit cluster_model to X
    cluster_model.fit(X)
    #Get the inertia value
    inertia_value = cluster_model.inertia_
    #Append the inertia_value to inertia list
    inertia.append(inertia_value)
##Inertia plot
plt.plot(range(1, 10), inertia)
plt.title('The Elbow Method - Inertia plot', fontsize = 20)
plt.xlabel('No. of Clusters')
plt.ylabel('inertia')
plt.show()

**K-Means**

Pada proses ini mendefinisikan algoritma Machine Learning K-Means dengan nilai K=4 dan menyimpan hasil kuslter kedalam kolom baru yaitu kolom 'Cluster'

In [ ]:
k = 4
kmeans = KMeans(n_clusters=k, random_state=0).fit(data_scaled)
df['Cluster'] = kmeans.labels_
df['Cluster'] = df['Cluster'].astype('category')

In [ ]:
df

**Persebaran data Kluster**

Pad proses ini menampilkan persebaran data setelah dilakukan klastering yaitu jumlah data per klaster dan persentasenya.

In [ ]:
cluster_size = df.groupby(['Cluster'], as_index=False).size()
cluster_size['Percentage'] = cluster_size['size'] / sum(cluster_size['size'])
cluster_size

**Deskripsi Data Setelah Kluster**

Pada proses ini menampilkan persebaran data deskriptif berdasarkan kluster

In [ ]:
df.groupby('Cluster').describe()

**Binning**

Pada proses ini melakukan binning data age/umur dengan membagi pada 3 bagian yaitu remaja, dewasa dan lansia

In [ ]:
age_labels = ['Remaja', 'Dewasa', 'Lansia']

# Code here
df['Age label'] = pd.qcut((df['Age']), 3, labels=age_labels)

**Grouping**

Pada proses ini melakukan grouping berdasarkan kolom Age yang sudah dilakukan binning

In [ ]:
cols = ['Age']

# Code here
groupby_age = df.groupby('Age label')[cols].mean()
groupby_age['Median'] = df.groupby('Age label')[cols].median()
groupby_age['Max'] = df.groupby('Age label')[cols].max()
groupby_age['Min'] = df.groupby('Age label')[cols].min()


groupby_age

Dari tabel diatas didapatkan :
* Untuk kelompok remaja rataannya pada umur 25, Nilai tengah 25, Nilai maksimal 28, Nilai Minimal 19.
* Untuk kelompok dewasa rataannya pada umur 33, Nilai tengah 33, Nilai maksimal 38, Nilai Minimal 29.
* Untuk kelompok dewasa rataannya pada umur 49, Nilai tengah 47, Nilai maksimal 75, Nilai Minimal 39.

**Dataframe Akhir**

Menampilkan hasil dataframe akhir setelah melakukan binning yang disimpan pada kolom 'Age label' dan klasterisasi yang disimpan pada kolom 'Cluster'

In [ ]:
df

# Hasil Klasterisasi

Cluster 0

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Age label', data = df[df['Cluster']==0])

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Duration', data = df[df['Cluster']==0])

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(16,15))
plt.tight_layout(4)

sns.countplot(x ='Sex', data = df[df['Cluster']==0], ax=ax[0][0])
sns.countplot(x ='Job', data = df[df['Cluster']==0], ax=ax[0][1])
sns.countplot(x ='Housing', data = df[df['Cluster']==0], ax=ax[0][2])
sns.countplot(x ='Saving accounts', data = df[df['Cluster']==0], ax=ax[1][0])
sns.countplot(x ='Checking account', data = df[df['Cluster']==0], ax=ax[1][1])
sns.countplot(x ='Purpose', data = df[df['Cluster']==0], ax=ax[1][2].tick_params(labelrotation=45))
plt.show()

Cluster 1

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Age label', data = df[df['Cluster']==1])

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Duration', data = df[df['Cluster']==1])

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(16,15))
plt.tight_layout(4)

sns.countplot(x ='Sex', data = df[df['Cluster']==1], ax=ax[0][0])
sns.countplot(x ='Job', data = df[df['Cluster']==1], ax=ax[0][1])
sns.countplot(x ='Housing', data = df[df['Cluster']==1], ax=ax[0][2])
sns.countplot(x ='Saving accounts', data = df[df['Cluster']==1], ax=ax[1][0])
sns.countplot(x ='Checking account', data = df[df['Cluster']==1], ax=ax[1][1])
sns.countplot(x ='Purpose', data = df[df['Cluster']==1], ax=ax[1][2].tick_params(labelrotation=45))
plt.show()

Cluster 2

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Age label', data = df[df['Cluster']==2])

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Duration', data = df[df['Cluster']==2])

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(16,15))
plt.tight_layout(4)

sns.countplot(x ='Sex', data = df[df['Cluster']==2], ax=ax[0][0])
sns.countplot(x ='Job', data = df[df['Cluster']==2], ax=ax[0][1])
sns.countplot(x ='Housing', data = df[df['Cluster']==2], ax=ax[0][2])
sns.countplot(x ='Saving accounts', data = df[df['Cluster']==2], ax=ax[1][0])
sns.countplot(x ='Checking account', data = df[df['Cluster']==2], ax=ax[1][1])
sns.countplot(x ='Purpose', data = df[df['Cluster']==2], ax=ax[1][2].tick_params(labelrotation=45))
plt.show()

Cluster 3

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Age label', data = df[df['Cluster']==3])

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
sns.countplot(y ='Duration', data = df[df['Cluster']==3])

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(16,15))
plt.tight_layout(4)

sns.countplot(x ='Sex', data = df[df['Cluster']==3], ax=ax[0][0])
sns.countplot(x ='Job', data = df[df['Cluster']==3], ax=ax[0][1])
sns.countplot(x ='Housing', data = df[df['Cluster']==3], ax=ax[0][2])
sns.countplot(x ='Saving accounts', data = df[df['Cluster']==3], ax=ax[1][0])
sns.countplot(x ='Checking account', data = df[df['Cluster']==3], ax=ax[1][1])
sns.countplot(x ='Purpose', data = df[df['Cluster']==3], ax=ax[1][2].tick_params(labelrotation=45))
plt.show()

# Summary Hasil Analisis Data

* Terbagi menjadi 4 klaster dengan tipe yang berbeda.
* Nasabah kredit tersebar dari usia 19-75 tahun dan nasabah kredit bank paling banyak pada umur 25-30 tahun.      
* Nasabah kredit bank paling banyak pada tingkat pekerjaan 2 -Terlatih.
* Nasabah kredit bank paling banyak sudah memiliki rumah sendiri dan perbandingannya cukup dominan.
* Untuk gender dari semua cluster nasabah bank didominasi oleh laki-laki.

> Cluster 0
Didominasi pada label usia remaja, saving accountsnya
ada dari little-rich, memiliki credit amount dan durasi paling rendah, tujuan
utamanya didominasi untuk radio/tv, mobil. Pada cluster ini didominasi oleh
gender wanita.

> Cluster 1
Paling banyak nasabah pada cluster ini, didominasi pada label usia dewasa, saving accountsnya ada dari little-quite rich, memiliki credit
amount dan durasi kedua terendah, tujuan utamanya didominasi untuk radio/tv, mobil.

> Cluster 2
Paling sedikit nasabah di cluster ini, didominasi pada
label usia dewasa-lansia, job pada cluster ini sama dengan umumnya tapi diikuti oleh dominasi ke-2 yaitu level job 3-Professional, saving accountsnya ada dari little-rich, memiliki credit amount dan durasi paling tinggi dan jaraknya cukup jauh dengan cluster lainnnya, tujuan utamanya didominasi untuk mobil, bisnis, radio/tv.

> Cluster 3
Didominasi pada label usia lansia, saving accountsnya
ada dari quite rich-rich, chechking account nya ada dari unkown-little rich, memiliki credit amount dan durasi kedua terendah dengan tujuan utama yaitu untuk mobil, radio/tv, furniture.